In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [161]:
class Value:
    
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        self._prev = set(_children)
        self._op = _op
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, value):
        out = Value(self.data + value.data, (self, value), '+')
        return out
    
    def __mul__(self, value):
        out = Value(self.data * value.data, (self, value), '*')
        return out
    
    def __sub__(self, value):
        out = Value(self.data - value.data, (self, value), '-')
        return out
    
    def __truediv__(self, value):
        out = Value(self.data / value.data, (self, value), '/')
        return out
    
    def backward(self, earlier_derivative=None):
        
        if len(self._prev) > 0:
            child1, child2 = self._prev
            
            if self._op == '+':
                child1.grad = 1
                child2.grad = 1
                
            elif self._op == '*':
                child1.grad = child2.data
                child2.grad = child1.data
                
            elif self._op == '-':
                child1.grad = 1
                child2.grad = -1
                
            elif self._op == '/':
                child2.grad = 1 / child1.data
                child1.grad = - child2.data / (child1.data * child1.data)
                
            #elif self._op == '**':
            #    child1._grad 
            
            if earlier_derivative:
                child1.grad *= earlier_derivative
                child2.grad *= earlier_derivative
            
            if len(child1._prev) > 0:
                child1.backward(child1.grad)
                
            if len(child2._prev) > 0:
                child2.backward(child2.grad)
                

                
    def zero_grad(self):
        
        if len(self._prev) > 0:
            child1, child2 = self._prev
            child1.grad, child2.grad = 0, 0
            
            if len(child1._prev) > 0:
                child1.zero_grad()
                
            if len(child2._prev) > 0:
                child2.zero_grad()

In [162]:
a = Value(3)
b = Value(5)
c = Value(-3)
d = Value(1)

e = a * b
f = d + c

L = e * f


In [163]:
L.backward()

In [171]:
d.grad


15

In [144]:
f.grad

15

In [172]:
a = Value(3)
b = Value(6)

In [173]:
c = a / b

In [174]:
c

Value(data=0.5)

In [175]:
c.backward()

In [176]:
a.grad

0.16666666666666666

In [177]:
b.grad

-0.08333333333333333

In [121]:
-0.12*25

-3.0

In [122]:
-3/25

-0.12